<a href="https://colab.research.google.com/github/cartman12/DS-Unit-2-Classification-1/blob/master/module2-decision-trees/Mohamad%20Ali%20Nasser%20-%20Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The below is a work in progress! Hits is not in its finished form, will finish before class tomorrow.

In [8]:
!pip install category_encoders

In [9]:
!pip install pandas==0.23.4
!pip install graphviz
!apt-get install graphviz

     |████████████████████████████████| 8.9MB 4.8MB/s 
ERROR: google-colab 1.0.0 has requirement pandas~=0.24.0, but you'll have pandas 0.23.4 which is incompatible.
  Found existing installation: pandas 0.24.2
    Uninstalling pandas-0.24.2:
      Successfully uninstalled pandas-0.24.2


Reading package lists... Done
Building dependency tree       
Reading state information... Done
graphviz is already the newest version (2.40.1-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [10]:
%matplotlib inline
import category_encoders as ce
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
pd.set_option('display.float_format', '{:.2f}'.format)

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv('https://drive.google.com/uc?export=download&id=14ULvX0uOgftTB2s97uS8lIx1nHGQIB0P'), 
                 pd.read_csv('https://drive.google.com/uc?export=download&id=1r441wLr7gKGHGLyPpKauvCuUOU556S2f'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv('https://drive.google.com/uc?export=download&id=1wvsYl9hbRbZuIuoaLWCsW_kbcxCdocHz')
sample_submission = pd.read_csv('https://drive.google.com/uc?export=download&id=1kfJewnmhowpUo381oSn3XqsQ6Eto23XV')

# Split train into train & val
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=42)

# Print dataframe shapes
print('train', train.shape)
print('val', val.shape)
print('test', test.shape)

train (47520, 41)
val (11880, 41)
test (14358, 40)


In [0]:
def wrangle(X):
    """Wrangles train, validate, and test sets in the same way"""
    X = X.copy()
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace them with the column mean.
    cols_with_zeros = ['construction_year', 'longitude', 'latitude']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col] = X[col].fillna(X[col].mean())
        
    # Convert date_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract year from date_recorded
    X['year_recorded'] = X['date_recorded'].dt.year
    
    # quantity & quantity_group are duplicates, so drop one
    X = X.drop(columns='quantity_group')
    
    # for categoricals with missing values, fill with the category 'MISSING'
    categoricals = X.select_dtypes(exclude='number').columns
    for col in categoricals:
        X[col] = X[col].fillna('MISSING')
    
    return X


train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [12]:
train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,year_recorded
43360,72938,0.00,2011-07-27,MISSING,0,MISSING,33.54,-9.17,Kwa Mzee Noa,0,Lake Nyasa,Mpandapanda,Mbeya,12,4,Rungwe,Kiwira,0,True,GeoData Consultants Ltd,VWC,K,MISSING,1996.83,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe,functional,2011
7263,65358,500.00,2011-03-23,Rc Church,2049,ACRA,34.67,-9.31,Kwa Yasinta Ng'Ande,0,Rufiji,Kitichi,Iringa,11,4,Njombe,Imalinyi,175,True,GeoData Consultants Ltd,WUA,Tove Mtwango gravity Scheme,True,2008.00,gravity,gravity,gravity,wua,user-group,pay monthly,monthly,soft,good,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional,2011
2486,469,25.00,2011-03-07,Donor,290,Do,38.24,-6.18,Kwasungwini,0,Wami / Ruvu,Kwedigongo,Pwani,6,1,Bagamoyo,Mbwewe,2300,True,GeoData Consultants Ltd,VWC,MISSING,False,2010.00,india mark ii,india mark ii,handpump,vwc,user-group,pay per bucket,per bucket,salty,salty,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional,2011
313,1298,0.00,2011-07-31,Government Of Tanzania,0,DWE,30.72,-1.29,Kwajovin 2,0,Lake Victoria,Kihanga,Kagera,18,1,Karagwe,Isingiro,0,True,GeoData Consultants Ltd,MISSING,MISSING,True,1996.83,other,other,other,vwc,user-group,never pay,never pay,soft,good,enough,shallow well,shallow well,groundwater,other,other,non functional,2011
52726,27001,0.00,2011-03-10,Water,0,Gove,35.39,-6.40,Chama,0,Internal,Mtakuj,Dodoma,1,6,Bahi,Nondwa,0,True,GeoData Consultants Ltd,VWC,Zeje,True,1996.83,mono,mono,motorpump,vwc,user-group,pay per bucket,per bucket,soft,good,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,functional,2011


In [0]:
sns.relplot(x='longitude', y='latitude', hue='status_group', 
            data=train, alpha=0.1);

In [0]:
categorical_features = ['public_meeting', 'management', 'payment', 'waterpoint_type',
 'permit',
 'quantity',
 'payment_type',
 'source_type',
 'water_quality',
 'basin',
 'extraction_type','installer','ward','lga']
# The status_group column is the target
target = 'status_group'

# Get a dataframe with all train columns except the target & id
train_features = train.drop(columns=[target, 'id'])

# Get a list of the numeric features
# numeric_features = train_features.select_dtypes(include='number').columns.tolist()
numeric_features = ['amount_tsh', 'latitude','longitude',
 'gps_height',
  'num_private',
 'population', 'region_code','year_recorded']

cardinality_list = [x for x in train_features.select_dtypes(exclude='number').columns.tolist() if x not in categorical_features]

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features[categorical_features].select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features

In [0]:
cardinality

In [6]:
# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

# Encoder: fit_transform on train, transform on val & test
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_test_encoded = encoder.transform(X_test)

# Scaler: fit_transform on train, transform on val & test
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_val_scaled = scaler.transform(X_val_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

# Model: Fit on train, score on val, predict on test
model = LogisticRegression(solver='lbfgs', multi_class='auto', n_jobs=-1)
model.fit(X_train_scaled, y_train)
print('Validation Accuracy', model.score(X_val_scaled, y_val))
y_pred = model.predict(X_test_scaled)

# Write submission csv file
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('submission-02.csv', index=False)

Validation Accuracy 0.7204545454545455


In [0]:
coefficients = pd.Series(model.coef_[0], X_train_encoded.columns)
plt.figure(figsize=(10,30))
coefficients.sort_values().plot.barh(color='grey');

In [0]:
categorical_features = ['public_meeting', 'management', 'payment', 'waterpoint_type',
 'permit',
 'quantity',
 'payment_type',
 'source_type',
 'water_quality',
 'basin',
 'extraction_type','installer','ward','lga','subvillage','basin','funder']
# The status_group column is the target
target = 'status_group'

# Get a dataframe with all train columns except the target & id
train_features = train.drop(columns=[target, 'id'])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()
numeric_features = ['amount_tsh', 'latitude','longitude',
 'gps_height', 'region_code','year_recorded']

cardinality_list = [x for x in train_features.select_dtypes(exclude='number').columns.tolist() if x not in categorical_features]

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features[categorical_features].select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features

In [14]:
train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq,first,last
recorded_by,47520,1,GeoData Consultants Ltd,47520,NaN,NaN
status_group,47520,3,functional,25807,NaN,NaN
permit,47520,3,True,31071,NaN,NaN
public_meeting,47520,3,True,40838,NaN,NaN
source_class,47520,3,groundwater,36638,NaN,NaN
quantity,47520,5,enough,26567,NaN,NaN
management_group,47520,5,user-group,42027,NaN,NaN
quality_group,47520,6,good,40598,NaN,NaN
waterpoint_type_group,47520,6,communal standpipe,27642,NaN,NaN
payment_type,47520,7,never pay,20287,NaN,NaN


In [6]:
X_train

,amount_tsh,latitude,longitude,gps_height,region_code,year_recorded,public_meeting,management,payment,waterpoint_type,permit,quantity,payment_type,source_type,water_quality,basin,extraction_type,basin
43360,0.00,-9.17,33.54,0,12,2011,True,vwc,never pay,communal standpipe,MISSING,insufficient,never pay,spring,soft,Lake Nyasa,gravity,Lake Nyasa
7263,500.00,-9.31,34.67,2049,11,2011,True,wua,pay monthly,communal standpipe,True,enough,monthly,spring,soft,Rufiji,gravity,Rufiji
2486,25.00,-6.18,38.24,290,6,2011,True,vwc,pay per bucket,hand pump,False,insufficient,per bucket,shallow well,salty,Wami / Ruvu,india mark ii,Wami / Ruvu
313,0.00,-1.29,30.72,0,18,2011,True,vwc,never pay,other,True,enough,never pay,shallow well,soft,Lake Victoria,other,Lake Victoria
52726,0.00,-6.40,35.39,0,1,2011,True,vwc,pay per bucket,communal standpipe,True,enough,per bucket,borehole,soft,Internal,mono,Internal
8558,0.00,-8.43,31.21,1295,15,2011,True,vwc,never pay,communal standpipe,True,insufficient,never pay,river/lake,soft,Lake Tanganyika,gravity,Lake Tanganyika
2559,20000.00,-3.34,36.70,1515,2,2013,True,vwc,pay monthly,communal standpipe multiple,True,insufficient,monthly,spring,soft,Pangani,gravity,Pangani
54735,0.00,-5.18,36.29,0,1,2011,True,vwc,pay per bucket,communal standpipe,False,enough,per bucket,borehole,soft,Internal,mono,Internal
25763,0.00,-8.93,32.88,0,12,2011,False,vwc,never pay,hand pump,False,enough,never pay,borehole,soft,Lake Rukwa,swn 80,Lake Rukwa
44540,0.00,-3.12,33.01,0,19,2011,True,vwc,pay monthly,other,True,enough,monthly,borehole,soft,Lake Victoria,submersible,Lake Victoria


In [7]:
# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

# Encoder: fit_transform on train, transform on val & test
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_test_encoded = encoder.transform(X_test)

# Scaler: fit_transform on train, transform on val & test
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_val_scaled = scaler.transform(X_val_encoded)
X_test_scaled = scaler.transform(X_test_encoded)




AttributeError: ignored

In [5]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=42, max_depth = 20)
model.fit(X_train_encoded, y_train)
print('Decision Tree')
print('Train Accuracy', model.score(X_train_encoded, y_train))
print('Validation Accuracy', model.score(X_val_encoded, y_val))

NameError: ignored

In [0]:
importances = pd.Series(model.feature_importances_, 
                        X_train_encoded.columns)


In [0]:
importances

In [0]:
new_features = importances.sort_values(ascending = False)
new_features = importances[:70]

In [0]:
plt.figure(figsize=(10,30))
importances.sort_values().plot.barh(color='grey');

In [0]:
new_features = list(new_features.index)

In [62]:
# Scaler: fit_transform on train, transform on val & test
scaler = RobustScaler()
X_train_scaled2 = scaler.fit_transform(X_train_encoded[new_features])
X_val_scaled2 = scaler.transform(X_val_encoded[new_features])
X_test_scaled2 = scaler.transform(X_test_encoded[new_features])

# Model: Fit on train, score on val, predict on test
model = DecisionTreeClassifier(random_state=42, max_depth = 20)
model.fit(X_train_scaled2, y_train)
print('Decision Tree')
print('Train Accuracy', model.score(X_train_scaled2, y_train))
print('Validation Accuracy', model.score(X_val_scaled2, y_val))
# Write submission csv file
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('submission-02.csv', index=False)

Decision Tree
Train Accuracy 0.9000631313131313
Validation Accuracy 0.7544612794612795


Validation Accuracy 0.5867003367003367


In [0]:
categorical_features = ['public_meeting', 'management', 'payment', 'waterpoint_type',
 'permit',
 'quantity',
 'payment_type',
 'source_type',
 'water_quality',
 'basin',
 'extraction_type','installer','ward','lga','subvillage','basin','funder']
# The status_group column is the target
target = 'status_group'

# Get a dataframe with all train columns except the target & id
train_features = train.drop(columns=[target, 'id'])

# Get a list of the numeric features
# numeric_features = train_features.select_dtypes(include='number').columns.tolist()
numeric_features = ['amount_tsh', 'latitude','longitude',
 'gps_height',
 'population', 'region_code','year_recorded']

cardinality_list = [x for x in train_features.select_dtypes(exclude='number').columns.tolist() if x not in categorical_features]

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features[categorical_features].select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features

In [14]:
# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

# Encoder: fit_transform on train, transform on val & test
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_test_encoded = encoder.transform(X_test)

# Scaler: fit_transform on train, transform on val & test
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_val_scaled = scaler.transform(X_val_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

# Model: Fit on train, score on val, predict on test
model = LogisticRegression(solver='lbfgs', multi_class='auto', n_jobs=-1)
model.fit(X_train_scaled, y_train)
print('Validation Accuracy', model.score(X_val_scaled, y_val))
y_pred = model.predict(X_test_scaled)

# Write submission csv file
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('submission-02.csv', index=False)

AttributeError: ignored

In [0]:
# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

# Encoder: fit_transform on train, transform on val & test
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_test_encoded = encoder.transform(X_test)

# Scaler: fit_transform on train, transform on val & test
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_val_scaled = scaler.transform(X_val_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

# Model: Fit on train, score on val, predict on test
model = LogisticRegression(solver='lbfgs', multi_class='auto', n_jobs=-1)
model.fit(X_train_scaled, y_train)
print('Validation Accuracy', model.score(X_val_scaled, y_val))
y_pred = model.predict(X_test_scaled)

# Write submission csv file
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('submission-02.csv', index=False)

In [0]:
ortan

In [23]:
importances.shape

(94,)

In [29]:
importances

extraction_type_other - mkulima/shinyanga   0.00
extraction_type_climax                      0.00
extraction_type_other - play pump           0.00
waterpoint_type_dam                         0.00
extraction_type_walimi                      0.00
water_quality_fluoride abandoned            0.00
dtype: float64